In [ ]:
!pip install -U scikit-learn
%tensorflow_version 2.x

Requirement already up-to-date: scikit-learn in /usr/local/lib/python3.6/dist-packages (0.23.2)


From features generate malignant or begnin percentages using logistic regression for the breast cancer dataset.

 Re-install scikit-learn. Default version is 0.22, upgrading to 0.23.2. Reason: new parameter made available to import the dataset as a pandas DataFrame.

In [ ]:
import pandas as pd
import sklearn
import tensorflow as tf
import tensorflow.compat.v2.feature_column as fc 
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

 Import required python libraries.

In [ ]:
dataset = load_breast_cancer(as_frame=True)
x_train, x_eval , y_train, y_eval = train_test_split(dataset.data, dataset.target, test_size = 0.3, random_state = 0)
 
print('x, y | {}, {}'.format(x_train.shape, y_train.shape))
print(dataset.target.value_counts())
print('212 - Malignant, 357 - Benign')

x, y | (398, 30), (398,)
1    357
0    212
Name: target, dtype: int64
212 - Malignant, 357 - Benign


Load the breast cancer dataset as a pandas Dataframe. After loading, split the dataset into train and evaluation datasets in the ratio 0.7:0.3.
Additionally check the x_train dataset shape to ensure that the 1st dimension is similar to the 1st dimension of the evaluation dataset.
 
The dataset has two target variables Malignant,Benign labelled as 1 and 0 respectively. We are interested in the target variable because we will check how well the model does in percentage in predicting the target variables.

In [ ]:
FEATURE_COLUMNS = []
NUMERICAL_COLUMNS = dataset.feature_names
for n in NUMERICAL_COLUMNS:
  FEATURE_COLUMNS.append(tf.feature_column.numeric_column(n, dtype = tf.float32))

 The x_train and x_eval dataset contain 30 columns that are being translated to feature columns which describe to the estimator model how each feature should be interpreted. In this case a tf.feature_column.numerical_column is used since the dataset's columns' values are numbers. Tensorflow additionally supports categorical data. However, in this case it will not be put into use.

In [ ]:
#Input function required by tf.estimator.LinearClassifier
def input_fn(train_data, labels, epochs = 40, shuffle = True, batch_size = 20):
  ds = tf.data.Dataset.from_tensor_slices((dict(train_data), labels))
  if shuffle:
    ds = ds.shuffle(1159)
  ds = ds.batch(batch_size).repeat(epochs)
  return ds
 
mean_cross = tf.feature_column.crossed_column(['mean compactness', 'mean area'], hash_bucket_size= 30)
smoothness_radius = tf.feature_column.crossed_column(['mean radius', 'mean smoothness'], hash_bucket_size=30)
concativity_x_points = tf.feature_column.crossed_column(['mean concavity', 'mean concave points'], hash_bucket_size=30)
crossed_columns = [mean_cross, smoothness_radius]
linear_estimator = tf.estimator.LinearClassifier(feature_columns=FEATURE_COLUMNS + crossed_columns)
linear_estimator.train(input_fn= lambda: input_fn(x_train, y_train))
result = linear_estimator.evaluate(input_fn= lambda: input_fn(x_eval, y_eval, shuffle=False))
predictions = list(linear_estimator.predict(input_fn= lambda: input_fn(x_eval, y_eval, shuffle=False)))
print('Model accuracy: {}'.format(result['accuracy']))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp7ylnd1yh', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initiali

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/base_layer_v1.py:1727: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmp7ylnd1yh/model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...
INFO:tensorflow:loss = 0.6931472, step = 0
INFO:tensorflow:global_step/sec: 258.73
INFO:tensorflow:loss = 1.2733376, step = 100 (0.392 sec)
INFO:tensorflow:global_step/sec: 577.857
INFO:tensorflow:loss = 0.39867023, step = 200 (0.172 sec)
INFO:tensorflow:global_step/sec: 548.36
INFO:tensorflow:loss = 0.6502207, step = 300 (0.181 sec)
INFO:tensorflow:global_step/sec: 564.089
INFO:tensorflow:loss = 0.342014, step = 400 (0.175 sec)
INFO:tensorflow:global

 This section an input function which returns a tf.data.Dataset object (possibly containing large dataset) This objects pipes the dataset to the model through steaming due to limited system resources in batches. If the model is training we shuffle the data and batch it in 20. 
Afterwards, a crossed column between different features is made to increase the accuracy of the model. Using a single column, the model will assess how this column contributes to the output. However, crossing columns takes into account how two columns together contribute to the output. For example how does mean concavity and mean concavity points contribute to the output, associated weights have to be generated for this. We noted that crossing columns do not always increase the model accuracy. The crossed columns made above increased the accuracy from 91 to 94 percent.
 
Lastly,we create and train a LinearClassifier model. After training the model, we evaluate it and from the results of the evaluation , accuracy and loss can be obtained. From evaluation we use the same evaluation dataset to make a prediction.

In [ ]:
location_index = 1
print(dataset.data.loc[location_index]) #should locate data from the x_eval dataset
print('Label assigned: {}'.format(dataset.target.loc[location_index]))
print('Model prediction: {}'.format(predictions[0]))

mean radius                  20.570000
mean texture                 17.770000
mean perimeter              132.900000
mean area                  1326.000000
mean smoothness               0.084740
mean compactness              0.078640
mean concavity                0.086900
mean concave points           0.070170
mean symmetry                 0.181200
mean fractal dimension        0.056670
radius error                  0.543500
texture error                 0.733900
perimeter error               3.398000
area error                   74.080000
smoothness error              0.005225
compactness error             0.013080
concavity error               0.018600
concave points error          0.013400
symmetry error                0.013890
fractal dimension error       0.003532
worst radius                 24.990000
worst texture                23.410000
worst perimeter             158.800000
worst area                 1956.000000
worst smoothness              0.123800
worst compactness        